In [ ]:
pip install pyqt5

In [ ]:
"""
Load AirQualityUCI Data
"""
import pandas as pd
import numpy as np
from pandas import datetime
from matplotlib import pyplot as plt

def parser(x):
    return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

input_file = './data/AirQualityUCI_refined.csv'

df = pd.read_csv(input_file,
                 index_col=[0],
                 parse_dates=[0],
                 date_parser=parser)


# Visualization setup
%matplotlib
from matplotlib import pyplot as plt
import seaborn; seaborn.set()  # set plot styles
%config InlineBackend.figure_format = 'svg'
plt.rcParams['figure.figsize'] = [10, 5]
plt.ion() # enable the interactive mode

In [ ]:
# Visualize the 'CO(GT)' variable
df['CO(GT)'].plot()

In [ ]:
# Linear interpolation
co = df['CO(GT)'].copy()
co = co.interpolate(inplace = True)

In [ ]:
# Vusualize original and imputed data
plt.plot(df['CO(GT)'], label='original', zorder = 2)
plt.plot(co, label = 'linear interpolation', zorder = 1)
plt.legend(loc='best')

In [ ]:
# Detecting outliers using Boxplot
plt.boxplot(co)
plt.title("Detecting outliers using Boxplot")
plt.xlabel('CO(GT)')

In [ ]:
# Calculate correleations between variables
corr_matrix = df.corr() # -1 ~ 1 correlation value
print(corr_matrix)

In [ ]:
# Choose the least correlated variable
rh = df['RH'].copy().interpolate() # Relative Humidity

In [ ]:
# Visualize a scatter plot(CO, RH)
plt.scatter(co, rh, s=12, c='black')
plt.xlabel('CO(GT)')
plt.ylabel('RH')
plt.title("Detecting outliers using Boxplot")

In [ ]:
# Choose the most correlated variable
nmhc = df['PT08.52(NMHC)'].copy().interpolate() # NMHC : Non-metanic Hydro Carbon

In [ ]:
# Visualize a scatter plot(CO, NMHC)
plt.scatter(co, nmhc, s=12, c='black')
plt.xlabel('CO(GT)')
plt.ylabel('PT08.S2(NMHC)')

In [ ]:
"""
IQR-base Outlier Detection
"""

# Q1, Q2(median), Q3
q1 = co.quantile(0.25)
median = co.quantile(0.5)
q3 = co.quantile(0.75)
print(q1, medain, q3)

In [ ]:
# IQR, upper_fence, lower_fence
iqr = q3-q1
upper_fence = q3 + 1.5*iqr
lower_fence = q1 - 1.5*iqr
print(iqr, upper_fence, lower_fence)

In [ ]:
# Filtering the outliers
outliers = co.loc[(co > upper_fence) | (co < lower_fence)]
print(outliers)

In [ ]:
# Mask for outliers
mask = co.index.isin(outliers.index)
mask[:50]

In [ ]:
plt.plot(co[~mask], label='normal', color='blue',
        marker='o', markersize=3, linestlye='None')
plt.plot(co[~mask], label='outliers', color='red',
        marker='x', markersize=3, linestlye='None')
plt.legend(loc='best')

In [ ]:
# Removing the outliers
co_refined = co.copy()
co_refined[mask] = np.nan
print(co_refined[mask])
co_refined.plot()

In [ ]:
# Linear interpolation for reconstructing outliers removed.
co_refined.interpolate(inplace=True)
co_refined.plot()

In [ ]:
"""
Detecting Outliers with Z-Scores
"""

# Visualize the distribution of the 'CO(GT)' variable
import seaborn as sns
sns.distplot(co)

In [ ]:
# mean, standard deviation
mean = np.mean(co)
std = np.std(co)
print(mean, std)

In [ ]:
# Calculate Z-scores for each
outliers = []
thres = 3

for i in co:
    z_score = (i-mean) / std
    if(np.abs(z_score) > thres):
        print(z_score)
        outliers.append(i)

In [ ]:
outliers = co.loc[np.abs((co-mean)/std) > 3]
outliers

In [ ]:
# Mask for outliers
mask = co.index.isin(outliers.index) 
mask[:50]

In [ ]:
# Comparison of distributions before/after outlier removal
sns.distplot(co, axlabel='CO(GT)', label='origianl')
sns.distplot(co[~mask], label = 'outliers removed')
plt.legend(loc='best')

In [ ]:
# Flooring and Capping
floor = co.quantile(0.1)
cap = co.quantile(0.9)

co.loc[co < floor] = floor
co.loc[co > cap] = cap

In [ ]:
# Visualize the result
co.plot()